In [1]:
!pip install torch==2.3.1 torchaudio==2.3.1 transformers==4.46.0 torchcodec==0.2.0 noisereduce==3.0.2 tensorflow==2.16.1

ERROR: Could not find a version that satisfies the requirement torch==2.3.1 (from versions: 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.3.1


In [2]:
!pip install uvicorn


   ---------------------------------------- 0/3 [h11]
   ---------------------------------------- 0/3 [h11]
   ---------------------------------------- 0/3 [h11]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   ------------- -------------------------- 1/3 [click]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- ------------- 2/3 [uvicorn]
   -------------------------- -------

In [2]:
pip install torch torchaudio transformers tensorflow scikit-learn numpy


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   - -------------------------------------- 4.2/110.9 MB 23.2 MB/s eta 0:00:05
   ---- ----------------------------------- 11.3/110.9 MB 28.3 MB/s eta 0:00:04
   ------ --------------------------------- 17.6/110.9 MB 29.3 MB/s eta 0:00:04
   --

In [3]:
!pip install librosa

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------- ----- 2.4/2.7 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.3 MB/s  0:00:00
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ------ --------------------------------- 5.8/38.1 MB 29.0 MB/s eta 0:00:02
   ------------ --------------------------- 11.5/38.1 MB 27.6 MB/s eta 0:00:01
   ----------------- ---------------------- 17.0/38.1 MB 26.9 MB/s eta 0:00:01
   ----------------------- ---------------- 22.8/38.1 MB 27.4 MB/s eta 0:00:01
   ----------------------------- ---------- 28.6/38.1 MB 27.4 MB/s eta 0:00:01
   --------------------------------- ------ 32.0/38.1 MB 25.4 MB/s eta 0:00:01
   ------------------------------------- -- 35.4/38.1 MB 24.2 MB/s eta 0:00:01
   ---------------------------------------  38.0/38.1 MB 23.3 MB/s eta 0:00:01
   ---------------------------------------- 38.1/38.1 MB 22.0 MB/s  0:00:01
   

In [4]:
!pip install resampy

   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------- ----------------------------- 0.8/3.1 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 12.4 MB/s  0:00:00


In [12]:
!pip install soundfile



In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torchaudio
import torch
import soundfile as sf
import os
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load models
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)

#          Feature Extraction

def extract_wav2vec_embedding(path):
    audio, sr = sf.read(path)

    if audio.ndim == 1:
        audio = np.expand_dims(audio, axis=0)

    waveform = torch.tensor(audio, dtype=torch.float32)

    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)

    input_values = processor(
        waveform.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt"
    ).input_values.to(device)

    with torch.no_grad():
        outputs = model(input_values)
        hidden_states = outputs.last_hidden_state

    return hidden_states.mean(dim=1).squeeze().cpu().numpy()


#          Dataset Loading

data_dir = r"C:\Users\Utilisateur\Final_Project_sentiment_analysis\Training_dataset\Emotions"

X, y = [], []

valid_files = 0
skipped_files = 0

for root, _, files in os.walk(data_dir):
    for file in files:
        if not file.endswith(".wav"):
            continue

        parts = file.split("-")

        # ---- FILTER: Only accept files with valid RAVDESS naming ----
        if len(parts) < 3:
            print(f"Skipping invalid filename: {file}")
            skipped_files += 1
            continue

        try:
            # Emotion field
            emotion = int(parts[2]) - 1

            # Extract embedding
            emb = extract_wav2vec_embedding(os.path.join(root, file))

            if emb.shape == (768,):
                X.append(emb)
                y.append(emotion)
                valid_files += 1

        except Exception as e:
            print("Error with file:", file, "->", e)
            skipped_files += 1

X = np.stack(X)
y = np.array(y)

print("Valid processed files:", valid_files)
print("Skipped files:", skipped_files)
print("Feature shape:", X.shape)


#          Classifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

y_cat = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, stratify=y_cat, test_size=0.2, random_state=42
)

model_nn = Sequential([
    Dense(512, activation='relu', input_shape=(768,)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='softmax')
])

model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_nn.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=32)

loss, acc = model_nn.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.4f}")


#          Save model locally

save_model_path = r"C:\Users\Utilisateur\Final_Project_sentiment_analysis\Model\emotion_model"
model_nn.export(save_model_path)


print("Model saved to:", save_model_path)


c:\Users\Utilisateur\Final_Project_sentiment_analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


c:\Users\Utilisateur\Final_Project_sentiment_analysis\.venv\Lib\site-packages\transformers\configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Skipping invalid filename: 1001_DFA_ANG_XX.wav
Skipping invalid filename: 1001_IEO_ANG_HI.wav
Skipping invalid filename: 1001_IEO_ANG_LO.wav
Skipping invalid filename: 1001_IEO_ANG_MD.wav
Skipping invalid filename: 1001_IOM_ANG_XX.wav
Skipping invalid filename: 1001_ITH_ANG_XX.wav
Skipping invalid filename: 1001_ITS_ANG_XX.wav
Skipping invalid filename: 1001_IWL_ANG_XX.wav
Skipping invalid filename: 1001_IWW_ANG_XX.wav
Skipping invalid filename: 1001_MTI_ANG_XX.wav
Skipping invalid filename: 1001_TAI_ANG_XX.wav
Skipping invalid filename: 1001_TIE_ANG_XX.wav
Skipping invalid filename: 1001_TSI_ANG_XX.wav
Skipping invalid filename: 1001_WSI_ANG_XX.wav
Skipping invalid filename: 1002_DFA_ANG_XX.wav
Skipping invalid filename: 1002_IEO_ANG_HI.wav
Skipping invalid filename: 1002_IEO_ANG_LO.wav
Skipping invalid filename: 1002_IEO_ANG_MD.wav
Skipping invalid filename: 1002_IOM_ANG_XX.wav
Skipping invalid filename: 1002_ITH_ANG_XX.wav
Skipping invalid filename: 1002_ITS_ANG_XX.wav
Skipping inva

c:\Users\Utilisateur\Final_Project_sentiment_analysis\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.3527 - loss: 1.7832 - val_accuracy: 0.1988 - val_loss: 1.9634
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5509 - loss: 1.1997 - val_accuracy: 0.3614 - val_loss: 1.7397
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6004 - loss: 1.0581 - val_accuracy: 0.3313 - val_loss: 1.5317
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6419 - loss: 0.9784 - val_accuracy: 0.4518 - val_loss: 1.4308
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6740 - loss: 0.8686 - val_accuracy: 0.4518 - val_loss: 1.3867
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7256 - loss: 0.7581 - val_accuracy: 0.5482 - val_loss: 1.1822
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7282 - loss: 0.7174 - val_accuracy: 0.5783 - val_loss: 1.0291
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7617 - loss: 0.6481 - val_accuracy: 0.

INFO:tensorflow:Assets written to: C:\Users\Utilisateur\Final_Project_sentiment_analysis\Model\emotion_model\assets


Saved artifact at 'C:\Users\Utilisateur\Final_Project_sentiment_analysis\Model\emotion_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 768), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  1647049964624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052201552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052200976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052200784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052200208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052200016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052201360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052202320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052202128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1647052202704: TensorSpec(shape=(), dtype=tf.resource

In [ ]:
import os
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from transformers import Wav2Vec2Processor, Wav2Vec2Model, logging
from tensorflow.keras.models import load_model


# CONFIG

# Path to your own audio folder 
data_dir = "/lakehouse/default/Files/Diabolocom_audio"

# Path to your trained classifier
model_path = "/lakehouse/default/Files/Model/emotion_model.h5"

# Suppress Hugging Face warnings
logging.set_verbosity_error()

# Device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Wav2Vec2 feature extractor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)

# Load classifier 
classifier = load_model(model_path, compile=False)

# Emotion labels 
emotion_labels = [
    "neutral", "calm", "happy", "sad",
    "angry", "fearful", "disgust", "surprised"
]


# HELPER FUNCTIONS

def extract_embedding(path):
    """Extract Wav2Vec2 embedding from an audio file."""
    waveform, sr = torchaudio.load(path)

    # Resample to 16kHz
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)

    # Prepare for model
    input_values = processor(
        waveform.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt"
    ).input_values.to(device)

    # Forward pass
    with torch.no_grad():
        outputs = wav2vec_model(input_values)
        hidden_states = outputs.last_hidden_state

    return hidden_states.mean(dim=1).squeeze().cpu().numpy(), waveform, sr

def analyze_audio_with_model(path):
    """Analyze one audio file: print info, predict emotion, plot results."""
    emb, waveform, sr = extract_embedding(path)
    duration = waveform.shape[1] / sr

    # Run classifier
    pred = classifier.predict(emb.reshape(1, -1))
    emotion = emotion_labels[np.argmax(pred)]
    conf = float(np.max(pred))

    # Print summary
    print(f"File: {os.path.basename(path)}")
    print(f" - Sample rate: {sr} Hz")
    print(f" - Channels: {waveform.shape[0]}")
    print(f" - Duration: {duration:.2f} seconds")
    print(f" - Predicted Emotion: {emotion} (conf {conf:.2f})")
    print("-" * 40)

    # Plot waveform
    plt.figure(figsize=(12, 3))
    plt.plot(np.linspace(0, duration, waveform.shape[1]), waveform[0].numpy())
    plt.title(f"Waveform: {os.path.basename(path)} | Emotion: {emotion}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()

    # Plot spectrogram
    spec = torchaudio.transforms.MelSpectrogram(sample_rate=sr)(waveform)
    spec_db = torchaudio.transforms.AmplitudeToDB()(spec)

    plt.figure(figsize=(12, 4))
    plt.imshow(spec_db[0].numpy(), origin="lower", aspect="auto", cmap="inferno")
    plt.title(f"Mel Spectrogram: {os.path.basename(path)} | Emotion: {emotion}")
    plt.xlabel("Frames")
    plt.ylabel("Mel bins")
    plt.colorbar(format="%+2.0f dB")
    plt.show()

    return emotion



# MAIN EXECUTION

# Collect all supported audio files
audio_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(data_dir)
    for f in files if f.lower().endswith((".wav", ".mp3", ".flac", ".m4a"))
]

# Handle missing files
if not audio_files:
    raise FileNotFoundError(f"No audio files found in {data_dir}. "
                            "Make sure you placed .wav, .mp3, .flac, or .m4a files there.")

# Analyze files 
for f in audio_files[:20]:
    analyze_audio_with_model(f)
